# Background
Often in an ESI-mass spectrum, you can see many peaks corresponding to different charge-states of the same molecule (i.e., with the same mass). Unfortunately, we only know the $m/z$ value of these peaks, and do not know the charge corresponding to each peak. While neighboring peaks should have a $\pm 1$ relationship, we do not know if the first peak (i.e., highest $m/z$ we can see) is the $z=+1$ peak or some other charge. Charge determination must be performed in order to figure out the charge of each peak, and thus determine the mass of the molecule. Advanced deconvolution techinques exist for this purpose (including max-entropy based techniques), but here I'll derive an accessible formula using maximum-likelihood estimation to calculate the charge and mass of the molecule simultaneously from several peak locations.

## Nomenclature
* A list of peaks $\{x_i\}$. This is just several $m/z$ values you have identified as belonging to the same series in a mass spectrum (i.e., via their common envelope).
* A reference index, $0$. Pick one, it doesn't matter which -- it could be the largest, or the first.
* The mass of the reference molecule, $m_0$. Drop the $0$ for ease of typing ($m$). 
* The charge of the reference molecule, $z_0$. Drop the $0$ for ease of typing ($z$). 
* $n_i$ is the index relative to the reference peak (i.e., $n_0=0$, $n_1=1$, $n_{-1}=-1, ...$).
* $\langle X \rangle \equiv \frac{1}{N} \sum_{i=1}^{N} X_i$

## Assumptions
Peaks are located at: $$x_i = \frac{m_0+n_i}{z_0+n_i}$$

# MLE Equations
Rearrange to find the 
$$x_i z + n_i (x_i - 1) = m.$$
Note that this implies we can transform the peaks into mass if we know the charge and index. If the we assume normal distributed errors, and a common, constant variance, we can write 
$$\mathcal{L} = \prod_i \mathcal{N}(x_i z+n_i (x_i-1) \vert m, \sigma^2).$$

We'll use the maximum-likelihood approach to estimate $m$ and $z$. But first, note that
$$ \ln \mathcal{L} \propto - \sum_i (x_i z+n_i (x_i-1)- m)^2$$

## Estimate $m$
$$0=\frac{d \ln \mathcal{L}}{d m} $$
$$0= \sum_i 2(x_i z+n_i(x_i-1)-m)(-1)$$
$$= \sum_i (x_i z+n_i(x_i-1)-m)$$
$$= \langle xz \rangle + \langle n(x-1)\rangle - m$$

and finally
$$\implies m = z \langle x\rangle + \langle n(x-1)\rangle$$


## Estimate $z$
$$0=\frac{d \ln \mathcal{L}}{d z} $$
$$0= \sum_i 2(x_i z+n_i(x_i-1)-m)(x_i)$$
$$= z \langle x^2\rangle +\langle  nx(x-1) \rangle -m\langle x\rangle$$
$$= z \langle x^2\rangle +\langle  nx(x-1) \rangle -z\langle x\rangle^2 - langle x\rangle\langle n(x-1)\rangle$$
and finally
$$\implies z = \frac{\langle x\rangle \langle (x-1)n\rangle - \langle x(x-1)n \rangle}{\langle x^2 \rangle - \langle x \rangle^2}$$

## Revisit $m$
Plugging in, we get 
$$\implies m = \left(\frac{\langle x\rangle \langle (x-1)n\rangle - \langle x(x-1)n \rangle}{\langle x^2 \rangle - \langle x \rangle^2} \right) \langle x\rangle + \langle n(x-1)\rangle$$

## Thoughts
Therefore, we have two separate equations that only depend on the values of $\{x\}$ and $\{n\}$ and we can estimate $m$ and $z$ from all the datapoints simultaneously. Note this is better (i.e., statistically speaking) than the typical approach of taking two neighboring peaks and doing:
$$ x_1=\frac{m+1}{z+1} \approx \frac{m}{z+1} \rightarrow \frac{1}{x_1}= \frac{z}{m} + \frac{1}{m} = \frac{1}{x_0} + \frac{1}{m}$$
$$\implies m\approx \frac{x_0x_1}{x_0-x_1}$$

# Example
Here's some Python code showing how to do this calculation. Note: it assumes you didn't skip any peaks (i.e., ...+14,15,17,18...); better to use a smaller peak list with no skipped peaks.

In [1]:
## Myoglobin peaks from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1853331/
x = [1305.0,1212,1131,1060,998,943,893,849,808,772,738,707,679,653]


import numpy as np
x = np.array(x)
x = np.sort(x)[::-1]

n = np.arange(x.size)

E_x = np.mean(x)
E_x2 = np.mean(x**2.)
E_xm1_n = np.mean((x-1.)*n)
E_x_xm1_n = np.mean(x*(x-1.)*n)

z = (E_x * E_xm1_n - E_x_xm1_n)/(E_x2 - E_x**2.)

m = z*E_x + E_xm1_n

print("Mass: %.3f"%(m))
print('Charge_0:',z)
print('----------------')
print('Peak (m/z), z')
for i in range(x.size):
	print(x[i], int(z+n[i]))

Mass: 16964.682
Charge_0: 13.000357233961779
----------------
Peak (m/z), z
1305.0 13
1212.0 14
1131.0 15
1060.0 16
998.0 17
943.0 18
893.0 19
849.0 20
808.0 21
772.0 22
738.0 23
707.0 24
679.0 25
653.0 26
